In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns',50)
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_curve,precision_score,auc,roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score

from sklearn.svm import SVC 
def blight_model():

    
    train_data=pd.read_csv("train.csv",encoding='ISO-8859-1')
#     train_data=train_data.drop(['non_us_str_code'],axis=1)
    test_data=pd.read_csv("test.csv",encoding='ISO-8859-1')
    train_data.set_index('ticket_id',inplace=True) 
#     train_data.dropna(subset=['compliance'])
#     length=len(train_data.columns)
    total_null=train_data.isnull().sum()
    null_per=(train_data.isnull().sum()/train_data.isnull().count())
#     train_data=np.where(null_per>0.5)
    null_data=pd.concat([total_null,null_per],axis=1,keys=['total','percent'])
#     full_data=[train_data,test_data]
    train_data.drop(null_data[null_data['percent']>0.5].index,axis=1,inplace=True)
    address_data=pd.read_csv('addresses.csv')
    latlons_data=pd.read_csv('latlons.csv')
    address_lat=pd.merge(address_data,latlons_data)
    address_lat.set_index('ticket_id',inplace=True)
    train_data2=train_data.join(address_lat)
    test_data2=test_data.set_index('ticket_id').join(address_lat)
#     train_data2.drop(labels='clean_up_cost',inplace=True,axis=1)
#     test_data2.drop('clean_up_cost',inplace=True,axis=1)
    columns=[ 'violator_name','inspector_name','clean_up_cost',
       'violation_street_number', 'violation_street_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'city',
       'state', 'zip_code', 'country']
    train_data2.drop(columns,inplace=True,axis=1)
    test_data2.drop(columns,inplace=True,axis=1)
    train_data2['total_payment']=train_data2['fine_amount']+train_data2['admin_fee']+train_data2['state_fee']+train_data2['late_fee']-train_data2['discount_amount']
    test_data2['total_payment']=test_data2['fine_amount']+test_data2['admin_fee']+test_data2['state_fee']+test_data2['late_fee']-test_data2['discount_amount']
#     train_data2['hearing_date']=datetime.datetime(train_data2['hearing_date'].astype(int))
#     hearing_date=datetime.strptime(train_data2['hearing_date'],'%Y-%m-%d %H:%M:%S')
#     ticket_date=datetime.strptime(train_data2['ticket_issued_date'],'%Y-%m-%d %H:%M:%S')
    
#     hearing_date1=datetime.strftime(hearing_date,'%Y-%m-%d %H:%M:%S')
#     ticket_date1=datetime.strftime(ticket_date,'%Y-%m-%d %H:%M:%S')
    def time_gap(hearing_date_str,ticket_date_str):
        if not hearing_date_str or type(hearing_date_str)!=str:return 73
        hearing_date=datetime.strptime(hearing_date_str,"%Y-%m-%d %H:%M:%S")
        ticket_date=datetime.strptime(ticket_date_str, "%Y-%m-%d %H:%M:%S")    
        diff=hearing_date-ticket_date
        return diff.days
        
    train_data2['time_gap']=train_data2.apply(lambda x:time_gap(x['hearing_date'],x['ticket_issued_date']),axis=1)
    test_data2['time_gap']=test_data2.apply(lambda x:time_gap(x['hearing_date'],x['ticket_issued_date']),axis=1)
    column_list=['hearing_date','ticket_issued_date','fine_amount','admin_fee','state_fee','late_fee','discount_amount']
    train_data2.drop(column_list,axis=1,inplace=True)
    test_data2.drop(column_list,axis=1,inplace=True)
    for i in range(len(train_data2.columns)):
        if len(train_data2[train_data2.columns[i]].unique()) < 250:
            train_data2[train_data2.columns[i]] = train_data2[train_data2.columns[i]] .astype('category')

    string_name=['agency_name','disposition']
    train_cat=pd.get_dummies(train_data2,columns=string_name)
    test_cat=pd.get_dummies(test_data2,columns=string_name)
    final_clean= [
        'balance_due',
        'compliance_detail',
        'payment_amount',
        'payment_status',
        'violation_code',
        'violation_description',
        'address']
    train_cat.drop(final_clean,axis=1,inplace=True)
#     test_cat.drop(final_clean,axis=1,inplace=True) 
    train_cat['lat'].fillna(73,inplace='true')

    train_cat['lon'].fillna(73,inplace='true')
    test_cat['lon'].fillna(73,inplace='true')
    test_cat['lat'].fillna(73,inplace='true')
    train_set=train_cat[(train_cat['compliance']==0)|(train_cat['compliance']==1)]
    list_remove=['violation_zip_code','non_us_str_code','violation_code','violation_description','grafitti_status','address']
    test_cat.drop(list_remove,axis=1,inplace=True)
    
    #training the model
    y_train=train_set['compliance']
    X_train=train_set.drop('compliance',axis=1)
    X_test=test_cat
    train_features=X_train
    train_features_set=set(train_features)
    for features in set(train_features):
        if features not in X_test:
            train_features_set.remove(features)
    train_features_list=list(train_features_set)
#     print(train_features_list)
    X_train=X_train[train_features_list]
#     return X_train
    X_test=X_test[train_features_list]
    for depth,features in zip([20,50,10],[4,12,4]):
        forest=RandomForestRegressor(n_estimators=100,max_features=features,max_depth=depth,random_state=1).fit(X_train,y_train)
        y_pred=forest.predict(X_test)
#     print(roc_auc_score(y_test,y_pred))
#     print("depth:{:.1f},features:{:.1f}".format(depth,features))
    return pd.DataFrame(forest.predict(X_test),index=X_test.index,columns=['Compliance']) 
#   prinroc_auc_score(y_test,y_pred)
    
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,Compliance
ticket_id,
284932,0.056310
285362,0.014907
285361,0.066428
285338,0.069822
285346,0.080204
285345,0.070543
285347,0.079578
285342,0.743262
285530,0.031302
